# Flask GUI for Graph Colouring Problem Solver


In [1]:
%pip install flask networkx matplotlib



Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from flask import Flask, render_template, request, jsonify
import networkx as nx
import matplotlib.pyplot as plt
import os
import io
import base64
from Backtracking import backtracking_colouring
from CulturalAlgorithm import CulturalAlgorithm

In [ ]:
from flask import Flask, render_template, request, jsonify
import networkx as nx
import matplotlib.pyplot as plt
import io
import base64
from Backtracking import backtracking_colouring
from CulturalAlgorithm import CulturalAlgorithm

app = Flask(__name__)

def calculate_conflicts(graph, individual):
    conflicts = 0
    n = len(graph)
    for i in range(n):
        for j in range(i + 1, n):
            if graph[i][j] == 1 and individual[i] == individual[j]:
                conflicts += 1
    return conflicts

def calculate_chromatic_number(solution):
    if solution is None:
        return 0
    return len(set(solution))

def plot_graph_step(n, edges, coloring):
    canvas = nx.Graph()
    for i in range(n):
        canvas.add_node(i)
    for u, v in edges:
        canvas.add_edge(u, v)

    pos = nx.spring_layout(canvas, seed=42)
    plt.figure(figsize=(6,6))

    color_map = ["#DA42A2", "#60CF60", "#4A6EB8", 'yellow', 'purple', 'orange', 'cyan', 'magenta', 'lime', 'red']
    node_colors = [color_map[c % len(color_map)] if c != -1 else '#888' for c in coloring]

    nx.draw(canvas, pos, with_labels=True, node_color=node_colors, node_size=700, edgecolors='black')
    
    buf = io.BytesIO()
    plt.tight_layout()
    plt.savefig(buf, format='png')
    buf.seek(0)
    img_str = base64.b64encode(buf.read()).decode('utf-8')
    plt.close()
    return img_str

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/simulate', methods=['POST'])
def simulate():
    data = request.json
    n = data['nodes']
    edges = data['edges']
    algo = data['algorithm']
    max_colours = data.get('max_colours', n)

    pop_size = data.get('pop_size', 20)
    generations = data.get('generations', 50)
    mutation_rate = data.get('mutation_rate', 0.1)

    graph = [[0]*n for _ in range(n)]
    for u, v in edges:
        graph[u][v] = 1
        graph[v][u] = 1

    steps = []

    try:
        if algo == 'backtracking':
            solution, exec_time = backtracking_colouring(graph, max_colours)
            if solution is not None:
                steps.append(solution.copy())
            final_solution = solution if solution else [-1]*n
            final_conflicts = calculate_conflicts(graph, final_solution)
            chromatic_number = calculate_chromatic_number(final_solution)
            return jsonify({
                "algorithm": "Backtracking",
                "solution": final_solution,
                "conflicts": final_conflicts,
                "chromatic_number": chromatic_number,
                "time": exec_time,
                "steps": steps
            })
        else:
            # Cultural Algorithm
            ca = CulturalAlgorithm(
                graph, 
                population_size=pop_size, 
                generations=generations, 
                mutation_rate=mutation_rate, 
                max_colours=max_colours
            )

            best_solution, best_fitness, exec_time = ca.evolve()
            steps.append(best_solution.copy())

            final_conflicts = calculate_conflicts(graph, best_solution)
            chromatic_number = calculate_chromatic_number(best_solution)

            return jsonify({
                "algorithm": "Cultural Algorithm",
                "solution": best_solution,
                "conflicts": final_conflicts,
                "chromatic_number": chromatic_number,
                "time": exec_time,
                "steps": steps
            })

    except Exception as e:
        print("Simulation error:", e)
        return jsonify({"error": str(e)}), 500

@app.route('/step_image', methods=['POST'])
def step_image():
    data = request.json
    n = data['nodes']
    edges = data['edges']
    coloring = data['coloring']
    img_str = plot_graph_step(n, edges, coloring)
    return jsonify({'image': img_str})

if __name__ == '__main__':
    app.run(debug=True, use_reloader=False)


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [13/Nov/2025 11:36:29] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [13/Nov/2025 11:39:03] "POST /simulate HTTP/1.1" 200 -
C:\Users\ok\AppData\Local\Temp\ipykernel_15100\3882745802.py:41: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout()
127.0.0.1 - - [13/Nov/2025 11:39:04] "POST /step_image HTTP/1.1" 200 -
127.0.0.1 - - [13/Nov/2025 11:39:04] "POST /step_image HTTP/1.1" 200 -
127.0.0.1 - - [13/Nov/2025 11:53:36] "POST /simulate HTTP/1.1" 200 -
127.0.0.1 - - [13/Nov/2025 11:53:36] "POST /step_image HTTP/1.1" 200 -
127.0.0.1 - - [13/Nov/2025 11:53:37] "POST /step_image HTTP/1.1" 200 -
127.0.0.1 - - [13/Nov/2025 11:58:00] "POST /simulate HTTP/1.1" 200 -
127.0.0.1 - - [13/Nov/2025 11:58:00] "POST /step_image HTTP/1.1" 200 -
127.0.0.1 - - [13/Nov/2025 11:58:01] "POST /step_image HTTP/1.1" 200 -
127.0.0.1 - - [13/Nov/2025 11:58:24] "POST /simulat